In [ ]:
from typing import TypedDict, Annotated, Sequence, List
from langchain_core.documents.base import Document

class AgentState(TypedDict):
    actions: str
    context: Sequence[Document]
    answer: str
    question: str
    groundness_status: str

In [ ]:
def extract_actions(state: AgentState) -> AgentState:
    """액션 추출"""
    actions = "질문에서 추출된 액션들"
    return AgentState(actions=actions)

def is_exist_actions(state: AgentState) -> str:
    """액션 존재 여부 체크"""
    # is_exist_actions, is_not_exist_actions
    is_exist_actions = "is_exist_actions"
    return is_exist_actions

def is_exist_docs(state: AgentState) -> str:
    """검색 문서 존재 여부 체크"""
    # is_exist_docs, is_not_exist_docs
    is_exist_docs = "is_exist_docs"
    return is_exist_docs

def retrieve(state: AgentState) -> AgentState:
    """문서 검색"""
    docs = [Document(page_content="검색된 문서 1"), Document(page_content="검색된 문서 2")]
    return AgentState(context=docs)

def invoke_llm_with_context(state: AgentState) -> AgentState:
    """context와 함께 LLM 호출"""
    answer = "LLM 답변"
    return AgentState(answer=answer)

def invoke_llm(state: AgentState) -> AgentState:
    """LLM 호출"""
    answer = "LLM 답변"
    return AgentState(answer=answer)
    
def check_groundedness(state: AgentState) -> AgentState:
    """검색 문서와 LLM 답변의 적합성 확인"""
    # grounded, notGrounded, notSure
    groundness_status = "grounded" 
    return AgentState(groundness_status=groundness_status)
    
def judgement(state: AgentState) -> str:
    """노드 재호출 혹은 종료 판단"""
    # retry_retrieve, retry_invoke_llm_with_context, END
    judgement = "END" 
    return judgement

In [ ]:
from langgraph.graph import StateGraph

workflow = StateGraph(AgentState)
END = "END"


In [ ]:
workflow.add_node("extract_actions", extract_actions)
workflow.add_node("retrieve", retrieve)
workflow.add_node("invoke_llm", invoke_llm)
workflow.add_node("invoke_llm_with_context", invoke_llm_with_context)
workflow.add_node("check_groundedness", check_groundedness)

In [ ]:
workflow.add_conditional_edges(
    "extract_actions",
    is_exist_actions,
    {
        "is_exist_actions": "retrieve",
        "is_not_exist_actions": "invoke_llm",
    }
)
workflow.add_conditional_edges(
    "retrieve",
    is_exist_docs,
    {
        "is_exist_docs": "invoke_llm_with_context",
        "is_not_exist_docs": "invoke_llm",
    }
)
workflow.add_edge("invoke_llm_with_context", "check_groundedness")
workflow.add_edge("invoke_llm", END)
workflow.add_conditional_edges(
    "check_groundedness",
    judgement,
    {
        "retry_retrieve": "retrieve",
        "retry_invoke_llm_with_context": "invoke_llm_with_context",
        "END": END
    }
)

In [ ]:
workflow.set_entry_point("extract_actions")

In [ ]:
app = workflow.compile()

In [ ]:
app.invoke(AgentState(question="내 질문"))